In [1]:
import os
import json
import tensorflow as tf
import numpy as np
from pycocotools.cocoeval import COCOeval

from detection.datasets import coco, data_generator
from detection.models.detectors import faster_rcnn

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [6]:
img_mean = (123.675, 116.28, 103.53)
# img_std = (58.395, 57.12, 57.375)
img_std = (1., 1., 1.)

val_dataset = coco.CocoDataSet('/workspace/shared_workspace/data/coco/', 'val',
                               flip_ratio=0,
                               pad_mode='fixed',
                               mean=img_mean,
                               std=img_std,
                               scale=(800, 1216))
print(len(val_dataset))


model = faster_rcnn.FasterRCNN(
    num_classes=len(val_dataset.get_categories()))


img, img_meta, bboxes, labels = val_dataset[0]
batch_imgs = tf.Variable(np.expand_dims(img, 0))
batch_metas = tf.Variable(np.expand_dims(img_meta, 0))
model.layers[0].trainable=False
_ = model((batch_imgs, batch_metas), training=False)
for layer in model.layers[0].layers[0].layers[80:]:
    if type(layer)!=BatchNormalization:
        layer.trainable=True


loading annotations into memory...
Done (t=0.82s)
creating index...
index created!
4952


In [7]:
from tensorflow.python.keras.layers.normalization import BatchNormalization
for layer in model.layers[0].layers[0].layers[80:]:
    if type(layer)!=BatchNormalization:
        layer.trainable=True

In [8]:
model.load_weights('/workspace/shared_workspace/tf-eager-fasterrcnn/rcnn_keras_resnet_50_stage_4.h5', by_name=True)

In [10]:
img, img_meta, _, _ = val_dataset[0]

In [13]:
proposals = model.simple_test_rpn(img, img_meta)
res = model.simple_test_bboxes(img, img_meta, proposals)

In [14]:
model.backbone()

{'rois': array([[ 41.82089 ,  61.422504, 117.689   , 103.92925 ],
        [170.83339 , 246.84862 , 233.08075 , 318.86612 ],
        [190.4907  , 270.03955 , 230.30336 , 312.7552  ],
        [371.7068  , 287.89755 , 389.42313 , 296.51343 ],
        [180.25293 , 281.556   , 224.87338 , 310.39014 ],
        [160.63518 , 241.84042 , 211.91447 , 299.6857  ]], dtype=float32),
 'class_ids': array([34, 10, 10,  1, 10, 10], dtype=int32),
 'scores': array([0.9927739 , 0.91881365, 0.8723484 , 0.85960287, 0.707494  ,
        0.33169097], dtype=float32)}

In [7]:
batch_size = 1

dataset_results = []
imgIds = []
for idx in range(100):
    if idx % 100 == 0:
        print(idx)

    img, img_meta, _, _ = val_dataset[idx]

    proposals = model.simple_test_rpn(img, img_meta)
    res = model.simple_test_bboxes(img, img_meta, proposals)
    
    image_id = val_dataset.img_ids[idx]
    imgIds.append(image_id)
    
    for pos in range(res['class_ids'].shape[0]):
        results = dict()
        results['score'] = float(res['scores'][pos])
        results['category_id'] = int(res['class_ids'][pos])
        y1, x1, y2, x2 = [float(num) for num in list(res['rois'][pos])]
        results['bbox'] = [x1, y1, x2 - x1 + 1, y2 - y1 + 1]
        results['image_id'] = image_id
        dataset_results.append(results)

0


In [8]:
with open('coco_val2017_detection_result.json', 'w') as f:
    f.write(json.dumps(dataset_results))

In [9]:
coco_dt = val_dataset.coco.loadRes('coco_val2017_detection_result.json')

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!


In [10]:
cocoEval = COCOeval(val_dataset.coco, coco_dt, 'bbox')

In [11]:
cocoEval.params.imgIds = imgIds

In [12]:
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.15s).
Accumulating evaluation results...
DONE (t=0.18s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.005
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.001
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.004
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.003
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.001
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.003
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.003
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.002
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100